# Predicting VFD in Features_smogned

### Notebook automatically generated from your model

Model Decision Tree (All Models), trained on 2021-12-07 19:20:24.

#### Generated on 2021-12-08 00:23:01.529009

prediction
This notebook will reproduce the steps for a REGRESSION on  Features_smogned.
The main objective is to predict the variable VFD

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [0]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [0]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [0]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {'columns': [{'name': 'StudyID_Int', 'type': 'bigint'}, {'name': '1-Methyl-histidine', 'type': 'double'}, {'name': '3-Hydroxy-tyrosine', 'type': 'double'}, {'name': '3-Methyl-histidine', 'type': 'double'}, {'name': '4-Aminobenzoic acid', 'type': 'double'}, {'name': '4-Aminobutyric acid', 'type': 'double'}, {'name': '4-Hydroxyproline', 'type': 'double'}, {'name': 'Alanine', 'type': 'double'}, {'name': 'Alanyl-alanine', 'type': 'double'}, {'name': 'α-Aminoadipic acid', 'type': 'double'}, {'name': 'α-Aminobutyric acid', 'type': 'double'}, {'name': 'Anserine', 'type': 'double'}, {'name': 'Arginine', 'type': 'double'}, {'name': 'Asparagine', 'type': 'double'}, {'name': 'Aspartic acid', 'type': 'double'}, {'name': 'β-Alanine', 'type': 'double'}, {'name': 'β-Aminobutyric acid', 'type': 'double'}, {'name': 'β-Aminoisobutyric acid', 'type': 'double'}, {'name': 'Carnosine', 'type': 'double'}, {'name': 'Citrulline', 'type': 'double'}, {'name': 'Cysteine', 'type': 'double'}, {'name': 'Cystine', 'type': 'double'}, {'name': 'Ethanolamine', 'type': 'double'}, {'name': 'γ-Glutamyl-ε-lysine', 'type': 'double'}, {'name': 'Glutamic acid', 'type': 'double'}, {'name': 'Glutamine', 'type': 'double'}, {'name': 'Glycine', 'type': 'double'}, {'name': 'Glycine-proline', 'type': 'double'}, {'name': 'Histamine', 'type': 'double'}, {'name': 'Histidine', 'type': 'double'}, {'name': 'Homocystine', 'type': 'double'}, {'name': 'Homoserine', 'type': 'double'}, {'name': 'Isoleucine', 'type': 'double'}, {'name': 'Leucine', 'type': 'double'}, {'name': 'Lysine', 'type': 'double'}, {'name': 'Methionine', 'type': 'double'}, {'name': 'Methionine sulfoxide', 'type': 'double'}, {'name': 'Ornithine', 'type': 'double'}, {'name': 'Phenylalanine', 'type': 'double'}, {'name': 'Pipecolic acid', 'type': 'double'}, {'name': 'Proline', 'type': 'double'}, {'name': 'Proline-hydroxyproline', 'type': 'double'}, {'name': 'Pyroglutamic acid', 'type': 'double'}, {'name': 'Sarcosine', 'type': 'double'}, {'name': 'Seleno-methionine', 'type': 'double'}, {'name': 'Serine', 'type': 'double'}, {'name': 'Thiaproline', 'type': 'double'}, {'name': 'Threonine', 'type': 'double'}, {'name': 'Tryptophan', 'type': 'double'}, {'name': 'Tyrosine', 'type': 'double'}, {'name': 'Valine', 'type': 'double'}, {'name': 'Sex', 'type': 'string'}, {'name': 'Race', 'type': 'string'}, {'name': 'Infection Status', 'type': 'string'}, {'name': 'ECMO', 'type': 'boolean'}, {'name': 'Age_years', 'type': 'double'}, {'name': 'PRISM', 'type': 'bigint'}, {'name': 'PELOD', 'type': 'bigint'}, {'name': 'VFD', 'type': 'bigint'}], 'userModified': False}

ml_dataset_handle = dataiku.Dataset('Features_smogned')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [0]:
ml_dataset = ml_dataset[['Glycine-proline', 'PELOD', 'Pipecolic acid', 'Histidine', 'VFD', 'Anserine', 'γ-Glutamyl-ε-lysine', 'Thiaproline', 'Tyrosine', 'Alanyl-alanine', 'β-Aminoisobutyric acid', 'Threonine', 'Methionine', 'α-Aminoadipic acid', 'Citrulline', 'PRISM', 'Histamine', '3-Hydroxy-tyrosine', 'Lysine', 'Age_years', '4-Aminobenzoic acid', 'Ornithine', '3-Methyl-histidine', '4-Hydroxyproline', 'Homoserine', 'α-Aminobutyric acid', 'Glutamic acid', 'Cysteine', 'Methionine sulfoxide', 'Serine', 'Sex', 'Proline', 'Proline-hydroxyproline', 'Carnosine', 'Leucine', 'Glycine', 'β-Alanine', 'Cystine', 'Arginine', 'Infection Status', 'Pyroglutamic acid', 'Race', 'Aspartic acid', '1-Methyl-histidine', 'Isoleucine', 'ECMO', 'Ethanolamine', 'Alanine', 'Phenylalanine', 'Tryptophan', 'Seleno-methionine', 'Valine', 'Sarcosine', 'Asparagine', 'β-Aminobutyric acid', '4-Aminobutyric acid', 'Homocystine', 'Glutamine']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [0]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = ['Sex', 'Infection Status', 'Race', 'ECMO']
numerical_features = ['Glycine-proline', 'PELOD', 'Pipecolic acid', 'Histidine', 'Anserine', 'γ-Glutamyl-ε-lysine', 'Thiaproline', 'Tyrosine', 'Alanyl-alanine', 'β-Aminoisobutyric acid', 'Threonine', 'Methionine', 'α-Aminoadipic acid', 'Citrulline', 'PRISM', 'Histamine', '3-Hydroxy-tyrosine', 'Lysine', 'Age_years', '4-Aminobenzoic acid', 'Ornithine', '3-Methyl-histidine', '4-Hydroxyproline', 'Homoserine', 'α-Aminobutyric acid', 'Glutamic acid', 'Cysteine', 'Methionine sulfoxide', 'Serine', 'Proline', 'Proline-hydroxyproline', 'Carnosine', 'Leucine', 'Glycine', 'β-Alanine', 'Cystine', 'Arginine', 'Pyroglutamic acid', 'Aspartic acid', '1-Methyl-histidine', 'Isoleucine', 'Ethanolamine', 'Alanine', 'Phenylalanine', 'Tryptophan', 'Seleno-methionine', 'Valine', 'Sarcosine', 'Asparagine', 'β-Aminobutyric acid', '4-Aminobutyric acid', 'Homocystine', 'Glutamine']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]') or (hasattr(ml_dataset[feature].dtype, 'base') and ml_dataset[feature].dtype.base == np.dtype('M8[ns]')):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

We renamed the target variable to a column named target

In [0]:
ml_dataset['__target__'] = ml_dataset['VFD']
del ml_dataset['VFD']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

#### Cross-validation strategy

The dataset needs to be split into 2 new sets, one that will be used for training the model (train set)
and another that will be used to test its generalization capability (test set)

This is a simple cross-validation strategy.

In [0]:
train, test = pdu.split_train_valid(ml_dataset, prop=0.8)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [0]:
drop_rows_when_missing = []
impute_when_missing = [{'feature': 'Glycine-proline', 'impute_with': 'MEAN'}, {'feature': 'PELOD', 'impute_with': 'MEAN'}, {'feature': 'Pipecolic acid', 'impute_with': 'MEAN'}, {'feature': 'Histidine', 'impute_with': 'MEAN'}, {'feature': 'Anserine', 'impute_with': 'MEAN'}, {'feature': 'γ-Glutamyl-ε-lysine', 'impute_with': 'MEAN'}, {'feature': 'Thiaproline', 'impute_with': 'MEAN'}, {'feature': 'Tyrosine', 'impute_with': 'MEAN'}, {'feature': 'Alanyl-alanine', 'impute_with': 'MEAN'}, {'feature': 'β-Aminoisobutyric acid', 'impute_with': 'MEAN'}, {'feature': 'Threonine', 'impute_with': 'MEAN'}, {'feature': 'Methionine', 'impute_with': 'MEAN'}, {'feature': 'α-Aminoadipic acid', 'impute_with': 'MEAN'}, {'feature': 'Citrulline', 'impute_with': 'MEAN'}, {'feature': 'PRISM', 'impute_with': 'MEAN'}, {'feature': 'Histamine', 'impute_with': 'MEAN'}, {'feature': '3-Hydroxy-tyrosine', 'impute_with': 'MEAN'}, {'feature': 'Lysine', 'impute_with': 'MEAN'}, {'feature': 'Age_years', 'impute_with': 'MEAN'}, {'feature': '4-Aminobenzoic acid', 'impute_with': 'MEAN'}, {'feature': 'Ornithine', 'impute_with': 'MEAN'}, {'feature': '3-Methyl-histidine', 'impute_with': 'MEAN'}, {'feature': '4-Hydroxyproline', 'impute_with': 'MEAN'}, {'feature': 'Homoserine', 'impute_with': 'MEAN'}, {'feature': 'α-Aminobutyric acid', 'impute_with': 'MEAN'}, {'feature': 'Glutamic acid', 'impute_with': 'MEAN'}, {'feature': 'Cysteine', 'impute_with': 'MEAN'}, {'feature': 'Methionine sulfoxide', 'impute_with': 'MEAN'}, {'feature': 'Serine', 'impute_with': 'MEAN'}, {'feature': 'Proline', 'impute_with': 'MEAN'}, {'feature': 'Proline-hydroxyproline', 'impute_with': 'MEAN'}, {'feature': 'Carnosine', 'impute_with': 'MEAN'}, {'feature': 'Leucine', 'impute_with': 'MEAN'}, {'feature': 'Glycine', 'impute_with': 'MEAN'}, {'feature': 'β-Alanine', 'impute_with': 'MEAN'}, {'feature': 'Cystine', 'impute_with': 'MEAN'}, {'feature': 'Arginine', 'impute_with': 'MEAN'}, {'feature': 'Pyroglutamic acid', 'impute_with': 'MEAN'}, {'feature': 'Aspartic acid', 'impute_with': 'MEAN'}, {'feature': '1-Methyl-histidine', 'impute_with': 'MEAN'}, {'feature': 'Isoleucine', 'impute_with': 'MEAN'}, {'feature': 'Ethanolamine', 'impute_with': 'MEAN'}, {'feature': 'Alanine', 'impute_with': 'MEAN'}, {'feature': 'Phenylalanine', 'impute_with': 'MEAN'}, {'feature': 'Tryptophan', 'impute_with': 'MEAN'}, {'feature': 'Seleno-methionine', 'impute_with': 'MEAN'}, {'feature': 'Valine', 'impute_with': 'MEAN'}, {'feature': 'Sarcosine', 'impute_with': 'MEAN'}, {'feature': 'Asparagine', 'impute_with': 'MEAN'}, {'feature': 'β-Aminobutyric acid', 'impute_with': 'MEAN'}, {'feature': '4-Aminobutyric acid', 'impute_with': 'MEAN'}, {'feature': 'Homocystine', 'impute_with': 'MEAN'}, {'feature': 'Glutamine', 'impute_with': 'MEAN'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    test = test[test[feature].notnull()]
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    test[feature['feature']] = test[feature['feature']].fillna(v)
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

We can now handle the categorical features (still using the settings defined in Models):

Let's dummy-encode the following features.
A binary column is created for each of the 100 most frequent values.

In [0]:
LIMIT_DUMMIES = 100

categorical_to_dummy_encode = ['Sex', 'Infection Status', 'Race', 'ECMO']

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in categorical_to_dummy_encode:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(train, categorical_to_dummy_encode)

def dummy_encode_dataframe(df):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, coerce_to_unicode(dummy_value))
            df[dummy_name] = (df[feature] == dummy_value).astype(float)
        del df[feature]
        print ('Dummy-encoded feature %s' % feature)

dummy_encode_dataframe(train)

dummy_encode_dataframe(test)

Let's rescale numerical features

In [0]:
rescale_features = {'Glycine-proline': 'AVGSTD', 'PELOD': 'AVGSTD', 'Pipecolic acid': 'AVGSTD', 'Histidine': 'AVGSTD', 'Anserine': 'AVGSTD', 'γ-Glutamyl-ε-lysine': 'AVGSTD', 'Thiaproline': 'AVGSTD', 'Tyrosine': 'AVGSTD', 'Alanyl-alanine': 'AVGSTD', 'β-Aminoisobutyric acid': 'AVGSTD', 'Threonine': 'AVGSTD', 'Methionine': 'AVGSTD', 'α-Aminoadipic acid': 'AVGSTD', 'Citrulline': 'AVGSTD', 'PRISM': 'AVGSTD', 'Histamine': 'AVGSTD', '3-Hydroxy-tyrosine': 'AVGSTD', 'Lysine': 'AVGSTD', 'Age_years': 'AVGSTD', '4-Aminobenzoic acid': 'AVGSTD', 'Ornithine': 'AVGSTD', '3-Methyl-histidine': 'AVGSTD', '4-Hydroxyproline': 'AVGSTD', 'Homoserine': 'AVGSTD', 'α-Aminobutyric acid': 'AVGSTD', 'Glutamic acid': 'AVGSTD', 'Cysteine': 'AVGSTD', 'Methionine sulfoxide': 'AVGSTD', 'Serine': 'AVGSTD', 'Proline': 'AVGSTD', 'Proline-hydroxyproline': 'AVGSTD', 'Carnosine': 'AVGSTD', 'Leucine': 'AVGSTD', 'Glycine': 'AVGSTD', 'β-Alanine': 'AVGSTD', 'Cystine': 'AVGSTD', 'Arginine': 'AVGSTD', 'Pyroglutamic acid': 'AVGSTD', 'Aspartic acid': 'AVGSTD', '1-Methyl-histidine': 'AVGSTD', 'Isoleucine': 'AVGSTD', 'Ethanolamine': 'AVGSTD', 'Alanine': 'AVGSTD', 'Phenylalanine': 'AVGSTD', 'Tryptophan': 'AVGSTD', 'Seleno-methionine': 'AVGSTD', 'Valine': 'AVGSTD', 'Sarcosine': 'AVGSTD', 'Asparagine': 'AVGSTD', 'β-Aminobutyric acid': 'AVGSTD', '4-Aminobutyric acid': 'AVGSTD', 'Homocystine': 'AVGSTD', 'Glutamine': 'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale
        test[feature_name] = (test[feature_name] - shift).astype(np.float64) / scale

#### Modeling

Before actually creating our model, we need to split the datasets into their features and labels parts:

In [0]:
X_train = train.drop('__target__', axis=1)
X_test = test.drop('__target__', axis=1)

y_train = np.array(train['__target__'])
y_test = np.array(test['__target__'])

Now we can finally create our model!

In [0]:
from sklearn.tree import DecisionTreeRegressor
clf =  DecisionTreeRegressor(
                splitter = 'best',
                max_depth = 5,
                min_samples_leaf = 1
            )

... And train the model

In [0]:
%time clf.fit(X_train, y_train)

Build up our result dataset

In [0]:
%time _predictions = clf.predict(X_test)
predictions = pd.Series(data=_predictions, index=X_test.index, name='predicted_value')

# Build scored dataset
results_test = X_test.join(predictions, how='left')
results_test = results_test.join(test['__target__'], how='left')
results_test = results_test.rename(columns= {'__target__': 'VFD'})

#### Results

You can measure the model's accuracy:

In [0]:
c =  results_test[['predicted_value', 'VFD']].corr()
print ('Pearson correlation: %s' % c['predicted_value'][1])

That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
